# Segmentation

## Let’s get to work!

In [1]:
# general imports:
import os
import importlib
from torch import nn, cuda
from torch import optim
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

In [2]:
# my imports (reload all modules):

from data.dataset import load_data_deep_crack
from models.unet_resnet34 import UNetResNet34
from models.unet_swin import UNetSwin
from models.unet import UNet
from training.metrics import DiceLoss
from training.train import train
from utils import save_model_files
from utils import log_training_result


import data.dataset
import models.unet_resnet34
import models.unet_swin
import models.unet
import training.metrics
import training.train
import utils
importlib.reload(data.dataset)
importlib.reload(models.unet_resnet34)
importlib.reload(models.unet_swin)
importlib.reload(models.unet)
importlib.reload(training.metrics)
importlib.reload(training.train)
importlib.reload(utils)

c:\Users\Csaba\miniconda3\envs\cenv2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Csaba\miniconda3\envs\cenv2\Lib\site-packages\albumentations\__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.7' (you have '2.0.5'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


<module 'utils' from 'D:\\01_AdatMat_Elte\\03_THESIS\\concrete_crack_thesis\\src\\utils.py'>

In [3]:
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
print(f'Number of visible CUDA devices: {cuda.device_count()}')
cuda.is_available()
# NOTE: when referring to a CUDA device, torch will only know about visible devices
#       so while currently we will be using gpu05
#       this will be device `cuda:0`

Number of visible CUDA devices: 1


True

#### Loading the data

In [4]:
data_source ='../../DeepCrack/dataset/DeepCrack'
train_image_dir = data_source + '/train_img'
train_mask_dir = data_source + '/train_lab'
test_image_dir = data_source + '/test_img'
test_mask_dir = data_source + '/test_lab'

In [5]:
train_dl, val_dl, train_dataset, val_dataset = load_data_deep_crack(train_image_dir, train_mask_dir, [0.8, 0.2])

This gives us 240 train and 60 validation samples.


With CPU: 4m 12s


With CUDA: 3m 6s

In [28]:
import gc
# Define experiments: (layers_to_unfreeze, learning_rate)
experiments = [
    (['encoder4'], 1e-4),                       # B: Unfreeze layer4
    (['encoder3', 'encoder4'], 1e-5),           # C: Unfreeze layers 3,4
    (['encoder2', 'encoder3', 'encoder4'], 1e-5),# D: Unfreeze layers 2,3,4
    (None, 1e-5),                                # E: Unfreeze all
]
nr_of_epochs = 1
nr_of_epochs_save = 1
load_from_folder = 'resnet_1'
for i, (layers, lr) in enumerate(experiments, start=1):
    folder = f"resnet_unfreze_{i}/"
    
    print(f"=== Training experiment {i} | Unfreezing: {layers if layers is not None else 'ALL'} | LR: {lr} ===")

    model = UNetResNet34(   img_channels = 3,
                mask_channels = 1,
                base_channel_size = 64)  

    loss = DiceLoss()
    model.freeze_encoder_layers()
    model.unfreeze_encoder_layers(layers)

    optimizer = optim.Adam(params=model.parameters(), lr=lr)

    # Load logs from previous training (optional)
    dice_idcs = list(np.load('../saved_models/'+load_from_folder+'/dice_idcs.npy'))
    epoch_dice_idcs = list(np.load('../saved_models/'+load_from_folder+'/epoch_dice_idcs.npy'))
    val_dice_idcs = list(np.load('../saved_models/'+load_from_folder+'/val_dice_idcs.npy'))
    train_loss = list(np.load('../saved_models/'+load_from_folder+'/train_loss.npy'))
    val_loss = list(np.load('../saved_models/'+load_from_folder+'/val_loss.npy'))
    epoch_durations = list(np.load('../saved_models/'+load_from_folder+'/epoch_durations.npy'))
    best_model_wts = {}

    # Load pretrained weights
    model.load_state_dict(torch.load('../saved_models/'+load_from_folder+'/model_state_epoch_200.pth', weights_only=True))

    # Log config
    log_training_result('../saved_models/training_log_2.csv', {
        "timestamp": pd.Timestamp.now(),
        "weights_file": folder,
        "epochs": nr_of_epochs,
        "learning_rate": lr,
        "batch_size": 4,
        "accum_scale": 4,
        "comment": "Unfreeze layer",
        "augmentation": "rotate+randomCrop",
        "unfrezed layers": layers if layers is not None else "ALL"
    })
    model.save_trainable_layers_to_file('../saved_models/' + folder + 'trainable_layers.txt')
    # Train
    train(
        model, loss, optimizer,
        train_dl, val_dl,
        num_epochs=nr_of_epochs,
        accum_scale=4,
        dice_idcs=dice_idcs,
        epoch_dice_idcs=epoch_dice_idcs,
        val_dice_idcs=val_dice_idcs,
        best_model_wts=best_model_wts,
        train_loss=train_loss,
        val_loss=val_loss,
        epoch_durations=epoch_durations,
        save_path='../saved_models/' + folder,
        n_epoch_save=nr_of_epochs_save
    )
    del model
    del optimizer
    torch.cuda.empty_cache()
    gc.collect()  # Python garbage collection
    torch.cuda.reset_peak_memory_stats()
    torch.cuda.empty_cache()

=== Training experiment 1 | Unfreezing: ['encoder4'] | LR: 0.0001 ===


D:\01_AdatMat_Elte\03_THESIS\concrete_crack_thesis\src\utils.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
c:\Users\Csaba\miniconda3\envs\cenv2\Lib\site-packages\torch\nn\modules\conv.py:549: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\Convolution.cpp:1037.)
  return F.conv2d(


Train loss:  0.0737821767727534
Epoch  1 . finished.
Validation loss:  0.1442841370900472
save files
Epoch 1/1 completed in 87.72 seconds
=== Training experiment 2 | Unfreezing: ['encoder3', 'encoder4'] | LR: 1e-05 ===


c:\Users\Csaba\miniconda3\envs\cenv2\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Csaba\miniconda3\envs\cenv2\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
D:\01_AdatMat_Elte\03_THESIS\concrete_crack_thesis\src\utils.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the con

Train loss:  0.07878949542840322
Epoch  1 . finished.
Validation loss:  0.1465183695157369
save files
Epoch 1/1 completed in 103.02 seconds
=== Training experiment 3 | Unfreezing: ['encoder2', 'encoder3', 'encoder4'] | LR: 1e-05 ===


c:\Users\Csaba\miniconda3\envs\cenv2\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Csaba\miniconda3\envs\cenv2\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
D:\01_AdatMat_Elte\03_THESIS\concrete_crack_thesis\src\utils.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the con

Train loss:  0.07910446425278982
Epoch  1 . finished.
Validation loss:  0.14656991958618165
save files
Epoch 1/1 completed in 122.71 seconds
=== Training experiment 4 | Unfreezing: ALL | LR: 1e-05 ===


c:\Users\Csaba\miniconda3\envs\cenv2\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Csaba\miniconda3\envs\cenv2\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
D:\01_AdatMat_Elte\03_THESIS\concrete_crack_thesis\src\utils.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the con

Train loss:  0.07750977079073589
Epoch  1 . finished.
Validation loss:  0.146306570370992
save files
Epoch 1/1 completed in 164.61 seconds


In [26]:
import gc
# Define experiments: (layers_to_unfreeze, learning_rate)
experiments = [
    (['encoder4'], 1e-4),                       # B: Unfreeze layer4
    (['encoder3', 'encoder4'], 1e-5),           # C: Unfreeze layers 3,4
    (['encoder2', 'encoder3', 'encoder4'], 1e-5),# D: Unfreeze layers 2,3,4
    (None, 1e-5),                                # E: Unfreeze all
]

for i, (layers, lr) in enumerate(experiments, start=1):
    folder = f"resnet_6_{i}/"
    
    print(f"=== Training experiment {i} | Unfreezing: {layers if layers is not None else 'ALL'} | LR: {lr} ===")

    model = UNetResNet34(   img_channels = 3,
                mask_channels = 1,
                base_channel_size = 64)  

    loss = DiceLoss()
    model.freeze_encoder_layers()
    model.unfreeze_encoder_layers(layers)
    model.print_trainable_layers()

=== Training experiment 1 | Unfreezing: ['encoder4'] | LR: 0.0001 ===
stem.0.weight: requires_grad=True
stem.0.bias: requires_grad=True
stem.1.weight: requires_grad=True
stem.1.bias: requires_grad=True
encoder.initial.0.weight: requires_grad=False
encoder.initial.1.weight: requires_grad=False
encoder.initial.1.bias: requires_grad=False
encoder.encoder1.0.conv1.weight: requires_grad=False
encoder.encoder1.0.bn1.weight: requires_grad=False
encoder.encoder1.0.bn1.bias: requires_grad=False
encoder.encoder1.0.conv2.weight: requires_grad=False
encoder.encoder1.0.bn2.weight: requires_grad=False
encoder.encoder1.0.bn2.bias: requires_grad=False
encoder.encoder1.1.conv1.weight: requires_grad=False
encoder.encoder1.1.bn1.weight: requires_grad=False
encoder.encoder1.1.bn1.bias: requires_grad=False
encoder.encoder1.1.conv2.weight: requires_grad=False
encoder.encoder1.1.bn2.weight: requires_grad=False
encoder.encoder1.1.bn2.bias: requires_grad=False
encoder.encoder1.2.conv1.weight: requires_grad=Fal

In [19]:
model = UNetResNet34(   img_channels = 3,
                mask_channels = 1,
                base_channel_size = 64)  
for name, param in model.encoder.named_parameters():
    print(name)
model.freeze_encoder_layers()
model.print_trainable_layers()
model.unfreeze_encoder_layers(['encoder1'])

c:\Users\Csaba\miniconda3\envs\cenv2\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Csaba\miniconda3\envs\cenv2\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


initial.0.weight
initial.1.weight
initial.1.bias
encoder1.0.conv1.weight
encoder1.0.bn1.weight
encoder1.0.bn1.bias
encoder1.0.conv2.weight
encoder1.0.bn2.weight
encoder1.0.bn2.bias
encoder1.1.conv1.weight
encoder1.1.bn1.weight
encoder1.1.bn1.bias
encoder1.1.conv2.weight
encoder1.1.bn2.weight
encoder1.1.bn2.bias
encoder1.2.conv1.weight
encoder1.2.bn1.weight
encoder1.2.bn1.bias
encoder1.2.conv2.weight
encoder1.2.bn2.weight
encoder1.2.bn2.bias
encoder2.0.conv1.weight
encoder2.0.bn1.weight
encoder2.0.bn1.bias
encoder2.0.conv2.weight
encoder2.0.bn2.weight
encoder2.0.bn2.bias
encoder2.0.downsample.0.weight
encoder2.0.downsample.1.weight
encoder2.0.downsample.1.bias
encoder2.1.conv1.weight
encoder2.1.bn1.weight
encoder2.1.bn1.bias
encoder2.1.conv2.weight
encoder2.1.bn2.weight
encoder2.1.bn2.bias
encoder2.2.conv1.weight
encoder2.2.bn1.weight
encoder2.2.bn1.bias
encoder2.2.conv2.weight
encoder2.2.bn2.weight
encoder2.2.bn2.bias
encoder2.3.conv1.weight
encoder2.3.bn1.weight
encoder2.3.bn1.bias
en

In [25]:
model.freeze_encoder_layers()
model.unfreeze_encoder_layers(["encoder1"])
model.print_trainable_layers()

stem.0.weight: requires_grad=True
stem.0.bias: requires_grad=True
stem.1.weight: requires_grad=True
stem.1.bias: requires_grad=True
encoder.initial.0.weight: requires_grad=False
encoder.initial.1.weight: requires_grad=False
encoder.initial.1.bias: requires_grad=False
encoder.encoder1.0.conv1.weight: requires_grad=True
encoder.encoder1.0.bn1.weight: requires_grad=True
encoder.encoder1.0.bn1.bias: requires_grad=True
encoder.encoder1.0.conv2.weight: requires_grad=True
encoder.encoder1.0.bn2.weight: requires_grad=True
encoder.encoder1.0.bn2.bias: requires_grad=True
encoder.encoder1.1.conv1.weight: requires_grad=True
encoder.encoder1.1.bn1.weight: requires_grad=True
encoder.encoder1.1.bn1.bias: requires_grad=True
encoder.encoder1.1.conv2.weight: requires_grad=True
encoder.encoder1.1.bn2.weight: requires_grad=True
encoder.encoder1.1.bn2.bias: requires_grad=True
encoder.encoder1.2.conv1.weight: requires_grad=True
encoder.encoder1.2.bn1.weight: requires_grad=True
encoder.encoder1.2.bn1.bias: r